In [ ]:
##read in data
import os
import pandas as pd
import re


def search_dir(path, path_list=[]):
    files= os.listdir(path) # 得到文件夹下的所有文件名称
    # print(files)
    for file in files: # 遍历该文件夹
        if os.path.isdir(path+"/"+file): # 是子文件夹
            search_dir(path+"/"+file)
        else: # 是文件
            
            #path = re.sub('(")',r'\\\1',path)
            #file = re.sub('(")',r'\\\1',file)
            path_list.append(path+"/"+file)
           
    return path_list
            
path = r"/Users/yijunwang/Documents/CSI_pallative_care/case_study2/weibo-search/结果文件" # 文件夹目录
path_list = search_dir(path)
pall_tweets = pd.DataFrame(columns = ["id","bid","user_id","用户昵称","微博正文","头条文章url","发布位置","艾特用户","话题","转发数",'评论数','点赞数','发布时间','发布工具','微博图片url','微博视频url','retweet_id','关键词'],
                           dtype=str)
for i in path_list:
            print(i)
            pall_tweets_sub = pd.read_csv(i)
            pall_tweets_sub['关键词'] = re.findall(r'"(.*?)"', i)[0]
            pall_tweets = pall_tweets.append(pall_tweets_sub)
            print(len(pall_tweets))


mask = pall_tweets['微博正文'].str.contains("丧亲关怀|临终关怀|临终护理|临终照护|医学预嘱|医疗预立意愿|哀伤关怀|哀伤咨询|哀伤辅导|姑息医学|姑息医疗|姑息治疗|宁养服务|宁养照护|安宁医生|安宁护士|安宁照护|安宁疗护|安宁疗法|安宁病房|尊严死|有尊严地离世|有尊严的离世|有尊严得离世|死的有尊严|死得有尊严|死地有尊严|濒死期照护|灵性照护|生前医嘱|生前预嘱|生命终末期|癌症晚期照护|癌症末期照护|终末期照护|缓和医学|缓和医疗|舒缓医学|舒缓治疗|舒缓疗护|预先医疗指示|预嘱|预立医疗指示|预立医疗照护计划|预设医疗指示|疼痛病房")
pall_tweets = pall_tweets[mask==True]
pall_tweets = pall_tweets.drop_duplicates(keep='first',subset=["id"]) 
pall_tweets = pd.concat([pall_tweets,pall_tweets['发布时间'].str.rsplit(" ", n=1, expand=True)],axis=1)
pall_tweets = pall_tweets.rename(columns={0:'发布日期',1:'发布时刻'})
pall_tweets['发布日期'] = pall_tweets['发布日期'].str.replace('-', '', regex=True)
pall_tweets.to_csv(path+"/all_tweets.csv")

pall_tweets_user = pall_tweets[["user_id","发布日期"]]
pall_tweets_user = pall_tweets_user.drop_duplicates(keep='first',subset=["user_id","发布日期"])
pall_tweets_user.to_csv(path+"/all_tweets_user.csv")



In [ ]:
##detect robots
import requests
from bs4 import BeautifulSoup
import json
import time
import random
from requests.adapters import HTTPAdapter


#params = {"starttime": "20221001", "endtime": "20221115"}

def get_one_page(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36",
        "Cookie": "ALF=1670099661; _T_WM=9d091e19d1dbe2f134e14ab2b03bc4a7; SCF=AvJX5kDMJbZyzrcmduwgBUZk4Ll4cHG3L-kIRaRwxFYHJjb3Qj3VfRQxnS6oMmnezlbRbMgvGjVLzrNvoZvO58g.; SUB=_2A25OYFkXDeRhGeNM7VIV9ybFyj6IHXVtq2dfrDV6PUJbktANLXTekW1NTho77wYpgiIv1OXkPFAIgf-3cfIznKch; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFJCjNQCSTBbQk7DFMApKA25NHD95Qfeoq7ShMR1K2EWs4Dqc_zi--fiKLFiKL8i--fi-isiKnci--fiKysiK.Ri--RiK.ciKnXi--4iK.Ri-z0i--fiKnfiKn4i--fiKysiK.Ri--fiKnNiKn4; SSOLoginState=1667508551; __bid_n=1847c91a1d596815934207"
    }
    sess = requests.Session()
    sess.mount('http://', HTTPAdapter(max_retries=5)) 
    sess.mount('https://', HTTPAdapter(max_retries=5))     
    sess.keep_alive = False
    response = requests.get(url, headers=headers, timeout=(360,360), verify=False)
    if response.status_code == 200:
        return response.text
    else:
        logger.info("被ban了，需要等待一段时间")
        sys.exit()
    response.close()
    


def is_robot(html):
        soup = BeautifulSoup(html, "html.parser")
        tags = soup.find_all("input",{"name":"mp"})
        if len(tags) != 0:
            if int(tags[0].get("value")) >2:
                return(True)
            else:
                return(False)
        else:
            return(False)

def find_robot(user_id,user_date):
    url = "https://weibo.cn/%s/profile?hasori=0&haspic=0&starttime=%s&endtime=%s&advancedfilter=1&page=1"%(user_id,user_date,user_date)
    html = get_one_page(url)
    robot_value = is_robot(html)
    return robot_value
    
def main():
        
        weibo_robot_df = pd.read_csv(path+"/all_tweets_user_3.csv")
        for i in range(len(weibo_robot_df)):
            robot_dict = {}
            robot_value = find_robot(weibo_robot_df.iloc[i,1],weibo_robot_df.iloc[i,2])
            robot_dict[str(weibo_robot_df.iloc[i,1])] = robot_value
            with open(path+"/robot_user.json", "r") as file:
                js_data = json.load(file)
            js_data.update(robot_dict )
            with open(path+"/robot_user.json","w") as f:
                json.dump(js_data, f)
            time.sleep(random.randint(1, 5))
        #robot_json = json.dumps(robot_dict)
        #robot_file = open()
        #robot_file.write(robot_json)
        #robot_file.close()
main()

In [50]:
#delete robot records, and sample a subset to label retrival accuracy
import json
robot_user = []
with open(path+'/robot_user.json','r') as a:
    result = json.load(a)
    for key, values in result.items():
        if values == True:
            robot_user.append(key)
robot_user = map(int,robot_user)
robot_user = list(set(robot_user))
print(len(robot_user))




pall_tweets_1 = pall_tweets[~pall_tweets['user_id'].isin(robot_user)]
pall_tweets_1.to_csv(path+"/all_tweets_robot_removal.csv")
pall_tweets_1_user = pall_tweets_1["user_id"]
pall_tweets_1_user = pall_tweets_1_user.drop_duplicates(keep='first')
with open(path+'/user_id_list.txt','w') as f:
    for i in range(len(pall_tweets_1_user)):
        user_id = pall_tweets_1_user.iloc[i]
        f.writelines(str(user_id)+"\n")
f.close()

pall_tweets_2 = pall_tweets_1.sample(n=2000,replace=False,axis=0)
pall_tweets_2.to_csv(path+"/all_tweets_robot_removal_2000.csv")

pall_tweets_3 = pall_tweets_2.sample(n=1000,replace=False,axis=0)
pall_tweets_3.to_csv(path+"/all_tweets_robot_removal_1000.csv")

In [81]:
#calculate the coherence of two reviewers

pall_tweets_2 = pd.read_csv(path+"/all_tweets_robot_removal_2000.csv",dtype=str)
pall_tweets_3 = pd.read_csv(path+"/all_tweets_robot_removal_1000.csv",dtype=str)
YN_transfer ={"Y":'yes',"N":'no'}
pall_tweets_3['是否相关']= pall_tweets_3['是否相关'].map(YN_transfer) 
pall_tweets_2['是否一致'] = (pall_tweets_2.merge(pall_tweets_3, on=['id','是否相关'], how='left', indicator=True)
             .eval('_merge =="both"')
             .astype(int))
coherence = float(pall_tweets_2["是否一致"].value_counts()[1]/len(pall_tweets_3))
print("the cohenrence between two reviewers is: "+str(coherence))

the cohenrence between two reviewers is: 0.926


In [ ]:
#data prepsrocessing
import pandas as pd

#read in posts table and users table
pall_tweets = pd.read_csv(path+"/all_tweets_robot_removal.csv", dtype=str)
pall_tweets.dropna(subset=['微博正文'],inplace=True)
print("There is a total of "+str(len(pall_tweets))+" posts in this dataset.")

path_list = [r"/Users/yijunwang/Documents/CSI_pallative_care/case_study2/weibo-crawler/weibo/users.csv",
            r"/Users/yijunwang/Documents/CSI_pallative_care/case_study2/weibo-crawler1/weibo/users.csv",
            r"/Users/yijunwang/Documents/CSI_pallative_care/case_study2/weibo-crawler2/weibo/users.csv",
            r"/Users/yijunwang/Documents/CSI_pallative_care/case_study2/weibo-crawler3/weibo/users.csv",
            r"/Users/yijunwang/Documents/CSI_pallative_care/case_study2/weibo-crawler4/weibo/users.csv"] # 文件夹目录
pall_users = pd.DataFrame(columns = ["用户id","昵称","性别","生日","所在地","学习经历","公司","注册时间","阳光信用","微博数","粉丝数","关注数","简介","主页","头像","高清头像","微博等级","会员等级","是否认证","认证类型","认证信息","上次记录微博信息"],
                           dtype=str)
for i in path_list:
            print(i)
            pall_users_sub = pd.read_csv(i,dtype=str)
            pall_users = pall_users.append(pall_users_sub)
            print(len(pall_users))
pall_users = pall_users.rename(columns={'用户id':'user_id'})
print("There is a total of "+str(len(pall_users))+" users in this dataset.")
pall_users.to_excel('/Users/yijunwang/Documents/CSI_pallative_care/case_study2/weibo-search/结果文件/all_users_robot_removal_new.xlsx')

#merge two table according to user_id
pall_tweets = pd.merge(pall_tweets, pall_users, how="left", on="user_id")
print(len(pall_tweets))
pall_tweets.to_excel('/Users/yijunwang/Documents/CSI_pallative_care/case_study2/weibo-search/结果文件/all_tweets&users_robot_removal_new.xlsx')


#tranform categorical variables
pall_tweets['阳光信用'] = pall_tweets['阳光信用'].fillna(0,inplace=False)
pall_tweets['阳光信用'] = pall_tweets['阳光信用'].replace({'信用一般': 1, '信用较好': 2, '信用极好': 3}) #0:NaN, 1:信用一般, 2:信用较好, 3:信用极好
pall_tweets['学习经历'] = pall_tweets['学习经历'].fillna(0,inplace=False)
pall_tweets['学习经历'][pall_tweets['学习经历'] != 0] = 1 #NaN:0, not NaN:1
pall_tweets['公司'] = pall_tweets['公司'].fillna(0,inplace=False)
pall_tweets['公司'][pall_tweets['公司'] != 0] = 1 #NaN:0, not NaN:1
pall_tweets['是否认证'] = pall_tweets['是否认证'].fillna(0,inplace=False)
pall_tweets['是否认证'] = pall_tweets['是否认证'].replace({'True': 1, 'False': 0}) #0:False, 1:True

from datetime import datetime
date_format = "%Y-%m-%d"
start_date = datetime.strptime('2023-01-01', date_format)
pall_tweets['注册时间'] = pall_tweets['注册时间'].map(lambda x:(start_date-datetime.strptime(x,date_format)).days,na_action="ignore")
pall_tweets['注册时间'] = pall_tweets['注册时间'].fillna(pall_tweets['注册时间'].mean(),inplace=False)
pall_tweets['注册时间'] = pd.to_numeric(pall_tweets['注册时间']).round(0).astype(int) # tranform registration date as numerical variable

date_format = "%Y%m%d"
start_date = datetime.strptime('20230101', date_format)
pall_tweets['发布时间'] = pall_tweets['发布日期'].map(lambda x:(start_date-datetime.strptime(x,date_format)).days,na_action="ignore")
pall_tweets['发布时间'] = pall_tweets['发布时间'].fillna(pall_tweets['发布时间'].mean(),inplace=False)
pall_tweets['发布时间'] = pd.to_numeric(pall_tweets['发布时间']).round(0).astype(int) # tranform post date as numerical variable

pall_tweets = pall_tweets.drop("头条文章url",axis=1)
pall_tweets['发布位置'] = pall_tweets['发布位置'].fillna(0,inplace=False)
pall_tweets['发布位置'][pall_tweets['发布位置'] != 0] = 1 #NaN:0, not NaN:1
pall_tweets['艾特用户'] = pall_tweets['艾特用户'].fillna(0,inplace=False)
pall_tweets['艾特用户'][pall_tweets['艾特用户'] != 0] = 1 #NaN:0, not NaN:1
pall_tweets['话题'] = pall_tweets['话题'].fillna(0,inplace=False)
pall_tweets['话题'][pall_tweets['话题'] != 0] = 1 #NaN:0, not NaN:1


#remove unnecesssray variables
pall_tweets = pall_tweets.drop("发布时间", axis=1)
pall_tweets = pall_tweets.drop("发布工具", axis=1)
pall_tweets = pall_tweets.drop("retweet_id", axis=1)
pall_tweets = pall_tweets.drop("发布时刻", axis=1)
pall_tweets = pall_tweets.drop("昵称", axis=1)
pall_tweets = pall_tweets.drop("生日", axis=1)
pall_tweets = pall_tweets.drop("所在地", axis=1)
pall_tweets = pall_tweets.drop("主页", axis=1)
pall_tweets = pall_tweets.drop("头像", axis=1)
pall_tweets = pall_tweets.drop("高清头像", axis=1)
pall_tweets = pall_tweets.drop("上次记录微博信息", axis=1)
print("existing varibales in this dataset: "+pall_tweets.columns)

#count the number of links, pictures and videos in one post
pall_tweets['微博图片url'] = pall_tweets['微博图片url'].fillna("",inplace=False)
import re
pattern_re = r'http.*?jpg'
pall_tweets['微博图片url'] = pall_tweets['微博图片url'].map(lambda x:len(re.findall(pattern_re,x)))
pall_tweets['微博图片url'] = pd.to_numeric(pall_tweets['微博图片url']).round(0).astype(int) 

pall_tweets['微博视频url'] = pall_tweets['微博视频url'].fillna("",inplace=False)
pattern_re = r'http.*?video'
pall_tweets['微博视频url'] = pall_tweets['微博视频url'].map(lambda x:len(re.findall(pattern_re,x)))
pall_tweets['微博视频url'] = pd.to_numeric(pall_tweets['微博视频url']).round(0).astype(int) 

pattern_re = r'O网页链接'
pall_tweets['网页url'] = pall_tweets['微博正文'].map(lambda x:len(re.findall(pattern_re,x)))

#calculate the engagement rate
pall_tweets.dropna(subset=['转发数'],inplace=True)
pall_tweets.dropna(subset=['评论数'],inplace=True)
pall_tweets.dropna(subset=['点赞数'],inplace=True)
pall_tweets.dropna(subset=['粉丝数'],inplace=True)
pall_tweets['转发数'] = pall_tweets['转发数'].astype(int) 
pall_tweets['评论数'] = pall_tweets['评论数'].astype(int) 
pall_tweets['点赞数'] = pall_tweets['点赞数'].astype(int) 
pall_tweets['粉丝数'] = pall_tweets['粉丝数'].astype(int)
pall_tweets = pall_tweets.drop(pall_tweets[pall_tweets['粉丝数'] == 0].index)
pall_tweets['engagement_rate'] = pall_tweets.apply(lambda x:float(float(x['转发数']+x['评论数']+x['点赞数'])/float(x['粉丝数']))*1000, axis=1)
pall_tweets['engagement_number'] = pall_tweets.apply(lambda x:x['转发数']+x['评论数']+x['点赞数'], axis=1)


#calculate sentiment socre
##remove emoji
last_index = pall_tweets.index.tolist()[-1]+1
import re
p = re.compile(u'['u'\U0001F300-\U0001F64F' u'\U0001F680-\U0001F6FF' u'\u2600-\u2B55 \U00010000-\U0010ffff]+')
for i in range(last_index):
    try:
        pall_tweets["微博正文"][i] = re.sub(p,'',pall_tweets["微博正文"][i])
    except:
        pass
#remove punchuations
for i in range(last_index):
    try:
        pall_tweets["微博正文"][i] = pall_tweets["微博正文"][i].replace(u'\xa0', '')
        pall_tweets["微博正文"][i] = pall_tweets["微博正文"][i].replace(u'\u2022', '')
        pall_tweets["微博正文"][i] = pall_tweets["微博正文"][i].replace(u'\u200d', '')
    except:
        pass
    
#remove links (web page, video and article link)
for i in range(last_index):
    try:
        pall_tweets["微博正文"][i] = pall_tweets["微博正文"][i].replace(u'O网页链接', '')
        pall_tweets["微博正文"][i] = re.sub(re.compile(u'L.*的微博视频'),'',pall_tweets["微博正文"][i])
        pall_tweets["微博正文"][i] = re.sub(re.compile(u'L.*的秒拍视频'),'',pall_tweets["微博正文"][i])
        pall_tweets["微博正文"][i] = pall_tweets["微博正文"][i].replace(u'\u3000', '')
    except:
        pass

#sentiment analysis
pall_tweets["sentiment"] = 0
pall_tweets["negative_prob"] = 0.5
pall_tweets["positive_prob"] = 0.5
pall_tweets["confidence"] = 0.5
from aip import AipNlp

APP_ID = '29820259'
API_KEY = '5h7hc4gy5HCaL03rO1IBLzP0'
SECRET_KEY = 'vGdxkaNF23xFDYR5zdOos0UGVRP3dkIZ'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)
for i in range(last_index):
    
    try:
        text = pall_tweets["微博正文"][i]
        result = client.sentimentClassify(text)
        sentiment  = result['items'][0]["sentiment"]
        negative_prob = result['items'][0]["negative_prob"]
        positive_prob = result['items'][0]["positive_prob"]
        confidence = result['items'][0]["confidence"]
        pall_tweets["sentiment"][i] = sentiment
        pall_tweets["negative_prob"][i] = negative_prob
        pall_tweets["positive_prob"][i] = positive_prob
        pall_tweets["confidence"][i] = confidence
    except:
        pall_tweets["sentiment"][i] = 1
        pall_tweets["negative_prob"][i] = 0.5
        pall_tweets["positive_prob"][i] = 0.5
        pall_tweets["confidence"][i] = 0.5

pall_tweets['发布位置'] = pall_tweets['发布位置'].astype(int) 
pall_tweets['艾特用户'] = pall_tweets['艾特用户'].astype(int) 
pall_tweets['话题'] = pall_tweets['话题'].astype(int) 
pall_tweets['学习经历'] = pall_tweets['学习经历'].astype(int) 
pall_tweets['公司'] = pall_tweets['公司'].astype(int) 
pall_tweets['微博数'] = pall_tweets['微博数'].astype(int) 
pall_tweets['关注数'] = pall_tweets['关注数'].astype(int) 
pall_tweets['微博等级'] = pall_tweets['微博等级'].astype(int) 
pall_tweets['会员等级'] = pall_tweets['会员等级'].astype(int) 
pall_tweets['认证类型'] = pall_tweets['认证类型'].astype(int) 
 
 

